In [2]:
import gluonbook as gb
from mxnet import nd
from mxnet.gluon import rnn
(corpus_indices, char_to_idx, idx_to_char,vocab_size) = gb.load_data_jay_lyrics()

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = gb.try_gpu()
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    def _three():
        return (_one((num_inputs, num_hiddens)),_one((num_hiddens, num_hiddens)),nd.zeros(num_hiddens, ctx=ctx))
    W_xz, W_hz, b_z = _three() # 更新⻔参数。
    W_xr, W_hr, b_r = _three() # 重置⻔参数。
    W_xh, W_hh, b_h = _three() # 候选隐藏状态参数。
    # 输出层参数。
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 创建梯度。
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [5]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [6]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + R * nd.dot(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [7]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [8]:
gb.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,vocab_size, ctx, corpus_indices, idx_to_char,char_to_idx, False, num_epochs, num_steps, lr,clipping_theta, batch_size, pred_period, pred_len,prefixes)

epoch 40, perplexity 152.932234, time 1.66 sec
 - 分开 我想你的让我 我想你的让我 我想你的让我不 我想你的让我不 我想你的让我不 我想你的让我不 我想你
 - 不分开 我想你的让我 我想你的让我 我想你的让我不 我想你的让我不 我想你的让我不 我想你的让我不 我想你
epoch 80, perplexity 31.515494, time 1.75 sec
 - 分开 我想想这样 我有 这样的我有你 一场个人剧 你的让我面动 让你在我不多 让你在我不多 让你在我不多
 - 不分开 爱你在我已经你 一场个人剧 你的完美栈人 让果我有你想你 一场个人剧 你的完美栈人 你爱在我遇见你
epoch 120, perplexity 4.741425, time 1.66 sec
 - 分开我 想要这样的路快幽默 说便说你已经很美  穿和的话不起 让你在我都睡着   穿了其实我有多 我说的
 - 不分开 我知不觉 你已经一个人 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我
epoch 160, perplexity 1.429167, time 1.57 sec
 - 分开我 没有一直实我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
 - 不分开到  我去回到你对找 也才 你想上久了吧? 我的认真败给黑色幽默 走过了很多地方 我来到伊斯坦堡 就
